In [1]:
import powerlawrs
import polars as pl
import numpy as np
import pandas as pd
import math

In [2]:
file = "../reference_data/blackouts.txt"

# polars and pandas do NOT do a good job detecting headers, do not rely on them.
df = pl.read_csv(file, has_header=False)
data = df.to_series()

## Stats module

In [3]:
powerlawrs.descriptive.mean(data)

253868.68246445496

In [4]:
powerlawrs.descriptive.variance(data, 1)

372476564023.59814

In [5]:
powerlawrs.random.random_choice(data, 3)

[2100000.0, 126000.0, 75000.0]

In [6]:
powerlawrs.random.random_uniform(3)

[0.5447915956899041, 0.9133083012936987, 0.37386011425014365]

In [7]:
# Define a standard normal CDF in Python
norm_cdf = lambda x: 0.5 * (1 + math.erf(x / math.sqrt(2.0)))

sorted_data = [-1.1, -0.5, 0.1, 0.2, 1.5]

# Call your Rust function, passing the Python function as an argument
(d_plus, d_minus, d_max) = powerlawrs.ks.ks_1sam_sorted(sorted_data, norm_cdf)

print(f"D+: {d_plus}")
print(f"D-: {d_minus}")
print(f"D max: {d_max}")

D+: 0.22074029056089706
D-: 0.13982783727702897
D max: 0.22074029056089706


## Util module

In [8]:
powerlawrs.util.linspace(0,10,5)

[0.0, 2.5, 5.0, 7.5, 10.0]

In [16]:
simparams_dict = powerlawrs.sim.calculate_sim_params(0.01, data, 230000)
simparams_dict

{'num_sims_m': 2500,
 'sim_len_n': 211,
 'n_tail': 59,
 'p_tail': 0.2796208530805687}

In [22]:
# convert simparams dict to rust struct
simparams_struct = powerlawrs.sim.PySimParams(**simparams_dict)

# use the struct as an argument
sim_data = powerlawrs.sim.generate_synthetic_datasets(data, 230000, simparams_struct, 1.27)

In [27]:
#Note the library does not yet impliment zeta distribution for discrete data. 
pl.from_numpy(np.array(sim_data))

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_174,column_175,column_176,column_177,column_178,column_179,column_180,column_181,column_182,column_183,column_184,column_185,column_186,column_187,column_188,column_189,column_190,column_191,column_192,column_193,column_194,column_195,column_196,column_197,column_198,column_199,column_200,column_201,column_202,column_203,column_204,column_205,column_206,column_207,column_208,column_209,column_210
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
40000.0,112000.0,232191.265584,46000.0,281976.810529,278438.489147,160000.0,60000.0,25000.0,40000.0,90000.0,75000.0,315664.922553,81000.0,133000.0,18819.0,489585.305111,14273.0,1.7505e6,234883.997512,50000.0,5300.0,112000.0,1.0088e7,166000.0,51000.0,160000.0,454578.47972,50000.0,1.4937e6,70000.0,55000.0,277435.697621,70000.0,114500.0,36073.0,100000.0,…,190000.0,55000.0,50000.0,158000.0,1000.0,100000.0,674851.996016,45000.0,1.6933e6,299811.347573,50000.0,18000.0,50000.0,451026.084441,147000.0,29900.0,50000.0,46000.0,51000.0,20000.0,15000.0,256586.040452,26334.0,200000.0,25000.0,765428.569715,50000.0,235400.226305,15000.0,71000.0,1000.0,130000.0,45000.0,312952.8814,92000.0,1646.0,12000.0
30001.0,19000.0,24506.0,145000.0,146000.0,100000.0,805964.715982,219000.0,238595.887717,60000.0,60000.0,267819.865495,145000.0,318891.468692,100000.0,1.2167e6,37000.0,59000.0,479343.425941,142000.0,35000.0,60000.0,200000.0,243309.133316,251630.339514,158000.0,1.2386e6,115000.0,239471.226446,5.3587e6,38500.0,359745.878015,95630.0,29000.0,1800.0,51000.0,1.4005e7,…,126000.0,20000.0,5300.0,58000.0,81000.0,203000.0,100000.0,335569.675698,379204.960255,114500.0,200000.0,241604.07514,142000.0,50000.0,342784.563783,25000.0,249560.698015,32000.0,112000.0,70000.0,29000.0,4150.0,239630.076372,32000.0,502334.900777,19000.0,46000.0,290699.534384,113200.0,238690.521458,160000.0,537023.797923,80000.0,50462.0,24506.0,92000.0,71000.0
580147.742733,263795.611794,24506.0,60000.0,450248.729019,18819.0,145000.0,282411.349211,2900.0,33000.0,29000.0,1.7243e6,2900.0,8000.0,14273.0,66005.0,11000.0,38500.0,50000.0,32000.0,40000.0,426575.819204,158000.0,18000.0,56000.0,434502.729325,50000.0,95630.0,6.7320e6,62000.0,207200.0,1.1679e6,36073.0,75000.0,258601.014421,160000.0,9000.0,…,25000.0,145000.0,442692.330398,106850.0,1800.0,50462.0,120000.0,50000.0,296466.288171,297139.808716,36073.0,20000.0,158000.0,56000.0,142000.0,258579.858275,9000.0,62000.0,50000.0,50000.0,63500.0,9000.0,355424.397879,200000.0,146000.0,614175.261621,206000.0,145000.0,58000.0,91000.0,989335.449264,50000.0,71000.0,66005.0,9000.0,74000.0,435959.463579
266106.713975,242797.52947,963820.574405,24000.0,56000.0,937984.836359,493586.863509,409474.880256,339069.537303,88000.0,203000.0,37000.0,166000.0,100000.0,81000.0,63500.0,8000.0,95000.0,269350.708578,15000.0,39500.0,95630.0,30500.0,7500.0,420572.551461,158000.0,15000.0,40911.0,200000.0,146000.0,504240.653243,342534.117669,92000.0,737689.103771,55000.0,51000.0,33000.0,…,114500.0,207200.0,113200.0,292193.127308,43000.0,88000.0,461124.203781,112000.0,56000.0,142000.0,30001.0,92000.0,312836.890606,412350.298544,126000.0,53000.0,65000.0,17000.0,36073.0,12000.0,5300.0,122000.0,1.6374e6,173000.0,100000.0,81000.0,200000.0,100000.0,232054.487686,32000.0,191000.0,18819.0,50000.0,264952.704781,66005.0,17000.0,412925.338943
242029.9859,130000.0,160000.0,25000.0,348153.770927,253290.941124,163000.0,25000.0,207200.0,1030

# TODO

In [8]:
# generate an alpha param for every x_min of the data
(x_min, alphas) = powerlawrs.find_alphas_fast(df.to_series())

AttributeError: module 'powerlawrs' has no attribute 'find_alphas_fast'

In [48]:
fit = powerlawrs.gof(df.to_series(), x_min, alphas)

In [36]:
fit.alpha

1.2726372198302858

In [37]:
fit.x_min

230000.0

In [38]:
fit.D

0.06067379629443781

In [39]:
fit.len_tail

59

In [41]:
h0 = powerlawrs.hypothesis_test(df.to_series(), 0.01, fit.alpha, fit.x_min, fit.D)

Generating M = 2500 simulated datasets of length n = 211 with tail size 59 and probability of the tail P(tail|data) = 0.2796208530805687


In [42]:
h0.gt

1937

In [43]:
h0.pval

0.7748

In [44]:
h0.total

2500